# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:
w_data = pd.read_csv('output_data/Weather_data.csv')
w_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,60.01,72.0,26.0,5.99,AU,1.615747e+09
1,faanui,-16.4833,-151.7500,82.65,74.0,100.0,16.13,PF,1.615747e+09
2,wembley,-31.9333,115.8000,69.01,37.0,0.0,17.27,AU,1.615747e+09
3,okhotsk,59.3833,143.3000,1.80,78.0,99.0,9.26,RU,1.615747e+09
4,itarema,-2.9248,-39.9167,83.59,66.0,41.0,12.93,BR,1.615747e+09
...,...,...,...,...,...,...,...,...,...
560,vieques,18.4250,-65.8329,82.99,69.0,75.0,11.50,PR,1.615747e+09
561,drovyanaya,51.5833,113.0333,19.40,30.0,100.0,29.08,RU,1.615747e+09
562,zuwarah,32.9312,12.0820,59.77,60.0,0.0,5.41,LY,1.615747e+09
563,moose factory,51.2585,-80.6094,3.20,55.0,40.0,13.80,CA,1.615747e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
# Configure gmaps
gmaps.configure(api_key=g_key)


# define weights
locations = w_data[["Lat", "Lng"]].astype(float)

humid = w_data['Humidity'].astype(float)


# Set up heatmap
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3
                                )

# Center the chart
fig = gmaps.figure(center = [0,0], zoom_level =2)

# Add heat layer
fig.add_layer(heat_layer)


# Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
# no clouds
no_clouds = w_data.loc[(w_data['Cloudiness'] < 1)]


# wind speed less than 10mph

ideal_wind = no_clouds.loc[(no_clouds['Wind Speed']<10)]


# Ideal Temp between 80 and 90

temp_80 = ideal_wind.loc[(ideal_wind['Max Temp']>80)]
ideal_cities_df = temp_80.loc[(temp_80['Max Temp']<90)]
ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,atar,20.5169,-13.0499,83.17,15.0,0.0,8.84,MR,1.615747e+09
20,buin,-33.7333,-70.7500,82.99,41.0,0.0,6.91,CL,1.615747e+09
45,santiago,-33.4569,-70.6483,84.00,41.0,0.0,6.91,CL,1.615747e+09
162,mecca,21.4267,39.8261,80.58,47.0,0.0,2.86,SA,1.615747e+09
218,arlit,18.7369,7.3853,81.07,13.0,0.0,7.11,NE,1.615747e+09
394,tanout,14.9709,8.8879,84.65,11.0,0.0,6.46,NE,1.615747e+09
474,paso de carrasco,-34.8603,-56.0522,80.60,44.0,0.0,6.91,UY,1.615747e+09
482,caravelas,-17.7125,-39.2481,82.69,62.0,0.0,8.90,BR,1.615747e+09
541,bilma,18.6853,12.9164,83.84,10.0,0.0,5.50,NE,1.615747e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [101]:
# Add hotel name to dataframe
hotel_df = pd.DataFrame(ideal_cities_df, columns=['City', 'Country', 'Lat', 'Lng', 'Hotel Name'])


hotel_df.index.name = 'Index'

hotel_df



,City,Country,Lat,Lng,Hotel Name
Index,,,,,
7,atar,MR,20.5169,-13.0499,
20,buin,CL,-33.7333,-70.7500,
45,santiago,CL,-33.4569,-70.6483,
162,mecca,SA,21.4267,39.8261,
218,arlit,NE,18.7369,7.3853,
394,tanout,NE,14.9709,8.8879,
474,paso de carrasco,UY,-34.8603,-56.0522,
482,caravelas,BR,-17.7125,-39.2481,
541,bilma,NE,18.6853,12.9164,


In [113]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



# geocoordinates
target_radius = 5000
target_type = "lodging"


# setup parameters
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
    
    
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    
    params['location'] = f'{lat},{lng}'
    
    response = requests.get(base_url, params=params).json()
 
    # record hotel names
    hotel_results = response['results']
   
    
    try:
    
        hotel_df.loc[index,'Hotel Name'] = hotel_results[0]['name']
        
        
        
    except:(KeyError, IndexError)

    
response = requests.get(base_url, params=params)
    
hotel_response = response.json()

hotel_df

,City,Country,Lat,Lng,Hotel Name
Index,,,,,
7,atar,MR,20.5169,-13.0499,Oumou Elghoura
20,buin,CL,-33.7333,-70.7500,Paihuen
45,santiago,CL,-33.4569,-70.6483,Che Lagarto Hostel Santiago
162,mecca,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
218,arlit,NE,18.7369,7.3853,Case de Passage De la Prefecture
394,tanout,NE,14.9709,8.8879,
474,paso de carrasco,UY,-34.8603,-56.0522,Hotel Sofitel Montevideo Casino Carrasco and Spa
482,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes
541,bilma,NE,18.6853,12.9164,Expeditions Ténére voyages


In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [119]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
              
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))